In [1]:
import pymc as pm
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats, integrate
from plotly.subplots import make_subplots
from dataclasses import dataclass
from typing import Optional

import warnings

warnings.filterwarnings('ignore')

 **Create variables and functions**

In [2]:
# Create container for variables

@dataclass
class Company:
  name: str
  color: str
  line_width: int
  font_size: int
  percent_forecast_achieved: Optional[np.ndarray] = None
  extrapolated_forecast: Optional[np.ndarray] = None



In [3]:
# Create formatting variables

SiriusCyb = Company('SiriusCyb','#4f992d',2,14)
CompCo1 = Company('CompCo1','#2bb6d4',1.25,10)
CompCo2 = Company('CompCo2','#C53A66',1.25,10)
CompCo3 = Company('CompCo3','#6953ac',1.25,10)

In [4]:
# Use Plotly (https://plotly.com/) to create scatter chart and table function

def create_scatter_and_table(companyList):

  tableValues = np.array([company.extrapolated_forecast for company in companyList])
  tableValues = tableValues.transpose()

  fig = make_subplots(
    rows=2,cols=1,
    vertical_spacing=0.00,
        specs=[[{"type": "scatter"}],
              [{"type": "table"}]])

  for company in companyList:
    fig.add_trace(go.Scatter(y=company.extrapolated_forecast,x=month,line={'color':company.color,'width':company.line_width},name=company.name),row=1,col=1)

  fig.update_layout(
    xaxis={'visible':False},
    yaxis = {'visible':False},
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    height=900,
    width=1500,
    legend={'x':.2, 'y':.85}
    )

  fig.add_vrect(x0=10.8, x1 = 10.9,line_width=3,
               #line_dash="dash",
               fillcolor="#e5e8e8",
               line_color="#e5e8e8",
               opacity=.35,
               row=1,
               col=1)

  fig.add_annotation(x=0, y=0,
            text='Month',
            showarrow=False,
            borderpad =0,
            height = 30,
            valign = 'bottom',
            font={'color':'#636D78','size':10}
            )

  fig.add_trace(go.Table(    header={'values':month,
                                'fill_color':'#636D78',
                                'font':{'color':'white'}},
                                cells={'values':tableValues,
                                       'fill_color':'white',
                                       'font_color':[[company.color for company in companyList]]
                                       }

                        ),row=2,col=1)


  return fig


In [5]:
# Create histogram function

#Find bin edges for histogram

def find_bins(_array):
  bin_edges = np.histogram_bin_edges(_array,bins='scott')
  n_bins = len(bin_edges)
  return bin_edges, n_bins

# Produce histogram with annotations

def produce_histogram(data, lowest_feasible, annotate_x, annotate_y, histnorm = 'probability'):


  bin_edges, n_bins = find_bins(data)

  fig = px.histogram(data,nbins=n_bins,
                      color_discrete_sequence=[SiriusCyb.color],
                      histnorm = histnorm)

  fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    height=500,
    width=1500,
    showlegend = False,
    xaxis={'title':{'text':'Units','font':{'size':11}}},
    yaxis={'title':{'text':'Probability','font':{'size':11}}})



  hist_ = np.histogram(data,bins=bin_edges)
  stats_hist = stats.rv_histogram(hist_)
  probability = integrate.quad(stats_hist.pdf,lowest_feasible,max(data))
  annotation_text = f'''Probability > {lowest_feasible:,d} = {round(probability[0]*100,0)}% '''

  fig.add_annotation(x=annotate_x, y=annotate_y,
            text=annotation_text,
            font={'color':SiriusCyb.color,'size':14},
            showarrow=False,
            )

  return fig

**Bill's Forecast**

In [6]:
# Bill's forecast

month = [str(i) for i in range(1,25)]

SiriusCyb_monthly_forecast = np.array([
                                80,   110,  140,        #Q1  Y1
                                170,  240,  310,        #Q2  Y1
                                380,  490,  600,        #Q3  Y1
                                710,  880,  1050,       #Q4  Y1
                                1220, 1460, 1700,       #Q1  Y2
                                1940, 2250, 2560,       #Q2  Y2
                                2870, 3250, 3630,       #Q3  Y2
                                4190, 4750, 5310        #Q4  Y2
                                ])

SiriusCyb.extrapolated_forecast = SiriusCyb_monthly_forecast

create_scatter_and_table([SiriusCyb])

In [7]:
# Build pymc model (https://www.pymc.io) and use it use it to produce our initial distribution

# Use Range Rule to estimate standard deviation based on Bill's inputs

bill_forecasted_value = 1050
bill_lowest_value_considered = 600
skew = -4
sigma = (bill_forecasted_value-bill_lowest_value_considered) / 4

rng = np.random.default_rng(666)

with pm.Model() as pymcModel:
  siriuscybForecastDistribution = pm.SkewNormal('siriuscybForecastDistribution',alpha=skew,mu=bill_forecasted_value,sigma=sigma)
  sample_prior = pm.sample_prior_predictive(10000, random_seed=rng)


data = np.array(sample_prior['prior']['siriuscybForecastDistribution'][0])

lowest_feasible_for_new_facility = 850
annotate_x = 800
annotate_y = .06

produce_histogram(data,lowest_feasible_for_new_facility,annotate_x,annotate_y)

**Forecast attainment percentages applied to SiriusCyb**

In [8]:

CompCo1.percent_forecast_achieved =   np.array([0.85, 0.83, 0.73, 0.68, 0.58, 0.42, 0.67,  0.57, 0.57, 0.63, 0.71, 0.75, 0.68, 0.83, 0.79, 0.68, 0.79, 0.84, 0.84, 0.87, 0.69, 0.91, 0.90, 0.81])
CompCo2.percent_forecast_achieved = np.array([0.74, 0.95, 0.93, 0.78, 0.55, 0.63, 0.66 , 0.57, 0.60, 0.65, 0.61, 0.69, 0.75, 0.7 , 0.78, 0.76, 0.77, 0.98, 0.97, 0.71, 0.71, 0.86, 0.98, 0.98])
CompCo3.percent_forecast_achieved =    np.array([0.71, 0.92, 0.83, 0.65, 0.61, 0.77, 0.73,  0.51, 0.76 , 0.69, 0.67, 0.63, 0.76, 0.63, 0.79, 0.77, 0.63, 0.71, 0.92, 0.65, 0.95, 0.82, 0.90, 1.  ])

CompCo1.extrapolated_forecast = SiriusCyb.extrapolated_forecast * CompCo1.percent_forecast_achieved
CompCo1.extrapolated_forecast = np.around(CompCo1.extrapolated_forecast,0)

CompCo2.extrapolated_forecast = SiriusCyb.extrapolated_forecast * CompCo2.percent_forecast_achieved
CompCo2.extrapolated_forecast = np.around(CompCo2.extrapolated_forecast,0)

CompCo3.extrapolated_forecast = SiriusCyb.extrapolated_forecast * CompCo3.percent_forecast_achieved
CompCo3.extrapolated_forecast = np.around(CompCo3.extrapolated_forecast,0)

companyList = [SiriusCyb,CompCo1,CompCo2,CompCo3]


create_scatter_and_table(companyList)

In [ ]:
# Update original model with extrapolated forecasts (Bayesian Inference)


observed = [company.extrapolated_forecast[11] for company in companyList[1:]]


with pymcModel:
  updated = pm.SkewNormal('updated',mu = siriuscybForecastDistribution,alpha = skew, sigma = sigma, observed = observed)
  updatedSample = pm.sample(draws=100000, random_seed=rng)

In [10]:
updatedDistData = np.concatenate(updatedSample['posterior']['siriuscybForecastDistribution'])

annotate_x = 750
annotate_y = .04

produce_histogram(updatedDistData,lowest_feasible_for_new_facility,annotate_x,annotate_y)